In [27]:
import os, sys, re
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import pandas as pd
import tensorflow as tf
from tensorflow import keras
print(sys.version_info)
for module in tf, mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)
tensorflow 2.1.0
matplotlib 3.2.0
numpy 1.18.1
pandas 1.0.1
sklearn 0.22.2.post1
tensorflow 2.1.0
tensorflow_core.keras 2.2.4-tf


In [58]:

"""
Load Dataset from File
"""
#读取User数据
users_title = ['UserID', 'Gender', 'Age', 'JobID', 'Zip-code']
users = pd.read_csv('./data/ml-1m/users.dat', sep='::', header=None, names=users_title, engine = 'python')
users = users.filter(regex='UserID|Gender|Age|JobID')
users_orig = users.values
#改变User数据中性别和年龄
gender_map = {'F':0, 'M':1}
users['Gender'] = users['Gender'].map(gender_map)

age_map = {val:ii for ii,val in enumerate(set(users['Age']))}
users['Age'] = users['Age'].map(age_map)

#读取Movie数据集
movies_title = ['MovieID', 'Title', 'Genres']
movies = pd.read_csv('./data/ml-1m/movies.dat', sep='::', header=None, names=movies_title, engine = 'python')
movies_orig = movies.values
#将Title中的年份去掉
pattern = re.compile(r'^(.*)\((\d+)\)$')

title_map = {val:pattern.match(val).group(1) for ii,val in enumerate(set(movies['Title']))}
movies['Title'] = movies['Title'].map(title_map)

#电影类型转数字字典
genres_set = set()
for val in movies['Genres'].str.split('|'):
    genres_set.update(val)

genres_set.add('<PAD>')
genres2int = {val:ii for ii, val in enumerate(genres_set)}

#将电影类型转成等长数字列表，长度是18
genres_map = {val:[genres2int[row] for row in val.split('|')] for ii,val in enumerate(set(movies['Genres']))}

for key in genres_map:
    for cnt in range(max(genres2int.values()) - len(genres_map[key])):
        genres_map[key].insert(len(genres_map[key]) + cnt,genres2int['<PAD>'])

movies['Genres'] = movies['Genres'].map(genres_map)

#电影Title转数字字典
title_set = set()
for val in movies['Title'].str.split():
    title_set.update(val)

title_set.add('<PAD>')
title2int = {val:ii for ii, val in enumerate(title_set)}

#将电影Title转成等长数字列表，长度是15
title_count = 15
title_map = {val:[title2int[row] for row in val.split()] for ii,val in enumerate(set(movies['Title']))}

for key in title_map:
    for cnt in range(title_count - len(title_map[key])):
        title_map[key].insert(len(title_map[key]) + cnt,title2int['<PAD>'])

movies['Title'] = movies['Title'].map(title_map)

#读取评分数据集
ratings_title = ['UserID','MovieID', 'ratings', 'timestamps']
ratings = pd.read_csv('./data/ml-1m/ratings.dat', sep='::', header=None, names=ratings_title, engine = 'python')
ratings = ratings.filter(regex='UserID|MovieID|ratings')

#合并三个表
data = pd.merge(pd.merge(ratings, users), movies)

#将数据分成X和y两张表
target_fields = ['ratings']
# features.index 为 ['UserID', 'MovieID', 'Gender', 'Age', 'JobID', 'Title', 'Genres']
features_pd, targets_pd = data.drop(target_fields, axis=1), data[target_fields]
print(features_pd.columns)
print(targets_pd.columns)
features = features_pd.values
targets_values = targets_pd.values


#嵌入矩阵的维度
embed_dim = 32

# features:Index(['UserID', 'MovieID', 'Gender', 'Age', 'JobID', 'Title', 'Genres'], dtype='object')
#用户ID个数6040
uid_max = max(features.take(0,1)) + 1
#性别个数2
gender_max = max(features.take(2,1)) + 1
#年龄类别个数7
age_max = max(features.take(3,1)) + 1 
#职业个数21
job_max = max(features.take(4,1)) + 1

#电影ID个数3952
movie_id_max = max(features.take(1,1)) + 1 
#电影类型个数19
movie_categories_max = max(genres2int.values()) + 1 
#电影名单词个数5216
movie_title_max = len(title_set) 

#对电影类型嵌入向量做加和操作的标志，考虑过使用mean做平均，但是没实现mean
combiner = "sum"

#电影名长度
sentences_size = title_count # = 15
#文本卷积滑动窗口，分别滑动2, 3, 4, 5个单词
window_sizes = {2, 3, 4, 5}
#文本卷积核数量
filter_num = 8

#电影ID转下标的字典，数据集中电影ID跟下标不一致，比如第5行的数据电影ID不一定是5
movieid2idx = {val[0]:i for i, val in enumerate(movies.values)}



# Number of Epochs
num_epochs = 5
# Batch Size
batch_size = 256

dropout_keep = 0.5
# Learning Rate
learning_rate = 0.0001
# Show stats for every n number of batches
show_every_n_batches = 20

save_dir = './save'

Index(['UserID', 'MovieID', 'Gender', 'Age', 'JobID', 'Title', 'Genres'], dtype='object')
Index(['ratings'], dtype='object')


In [77]:
def get_inputs():
    uid = tf.keras.layers.Input(shape=(1,), dtype='int32', name='uid')  
    user_gender = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_gender')  
    user_age = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_age') 
    user_job = tf.keras.layers.Input(shape=(1,), dtype='int32', name='user_job')

    movie_id = tf.keras.layers.Input(shape=(1,), dtype='int32', name='movie_id') 
    movie_categories = tf.keras.layers.Input(shape=(18,), dtype='int32', name='movie_categories') 
    movie_titles = tf.keras.layers.Input(shape=(15,), dtype='int32', name='movie_titles') 
    return uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles

def get_user_embedding(uid, user_gender, user_age, user_job):
    '''
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length)
    1. define matrix : [vocab_size, embed_dim]
    2. [1,2,3,4..], max_length * embedding_dim
    3. batch_size * max_length * embedding_dim
    '''
    uid_embed_layer = tf.keras.layers.Embedding(uid_max, embed_dim, input_length=1, name='uid_embed_layer')(uid)
    gender_embed_layer = tf.keras.layers.Embedding(gender_max, embed_dim // 2, input_length=1, name='gender_embed_layer')(user_gender)
    age_embed_layer = tf.keras.layers.Embedding(age_max, embed_dim // 2, input_length=1, name='age_embed_layer')(user_age)
    job_embed_layer = tf.keras.layers.Embedding(job_max, embed_dim // 2, input_length=1, name='job_embed_layer')(user_job)
    return uid_embed_layer, gender_embed_layer, age_embed_layer, job_embed_layer

def get_user_feature_layer(uid_embed_layer, gender_embed_layer, age_embed_layer, job_embed_layer):
    #第一层全连接
    uid_fc_layer = tf.keras.layers.Dense(embed_dim, name="uid_fc_layer", activation='relu')(uid_embed_layer)
    gender_fc_layer = tf.keras.layers.Dense(embed_dim, name="gender_fc_layer", activation='relu')(gender_embed_layer)
    age_fc_layer = tf.keras.layers.Dense(embed_dim, name="age_fc_layer", activation='relu')(age_embed_layer)
    job_fc_layer = tf.keras.layers.Dense(embed_dim, name="job_fc_layer", activation='relu')(job_embed_layer)

    #第二层全连接
    user_combine_layer = tf.keras.layers.concatenate([uid_fc_layer, gender_fc_layer, age_fc_layer, job_fc_layer], 2)  #(?, 1, 128)
    user_combine_layer = tf.keras.layers.Dense(200, activation='tanh')(user_combine_layer)  #(?, 1, 200)

    user_combine_layer_flat = tf.keras.layers.Reshape([200], name="user_combine_layer_flat")(user_combine_layer)
    return user_combine_layer, user_combine_layer_flat

def get_movie_id_embed_layer(movie_id):
    movie_id_embed_layer = tf.keras.layers.Embedding(movie_id_max, embed_dim, input_length=1, name='movie_id_embed_layer')(movie_id)
    return movie_id_embed_layer

def get_movie_categories_layers(movie_categories):
    movie_categories_embed_layer = tf.keras.layers.Embedding(movie_categories_max, embed_dim, input_length=18, name='movie_categories_embed_layer')(movie_categories)
    movie_categories_embed_layer = tf.keras.layers.Lambda(lambda layer: tf.reduce_sum(layer, axis=1, keepdims=True))(movie_categories_embed_layer)
#     movie_categories_embed_layer = tf.keras.layers.Reshape([1, 18 * embed_dim])(movie_categories_embed_layer)

    return movie_categories_embed_layer

def get_movie_cnn_layer(movie_titles):
    #从嵌入矩阵中得到电影名对应的各个单词的嵌入向量
    movie_title_embed_layer = tf.keras.layers.Embedding(movie_title_max, embed_dim, input_length=15, name='movie_title_embed_layer')(movie_titles)
    sp=movie_title_embed_layer.shape
    movie_title_embed_layer_expand = tf.keras.layers.Reshape([sp[1], sp[2], 1])(movie_title_embed_layer)
    #对文本嵌入层使用不同尺寸的卷积核做卷积和最大池化window_sizes = {2, 3, 4, 5}， 文本卷积核数量filter_num = 8， 嵌入矩阵的维度 embed_dim = 32
    pool_layer_lst = []
    for window_size in window_sizes:
        conv_layer = tf.keras.layers.Conv2D(filter_num, (window_size, embed_dim), 1, activation='relu')(movie_title_embed_layer_expand)
        maxpool_layer = tf.keras.layers.MaxPooling2D(pool_size=(sentences_size - window_size + 1 ,1), strides=1)(conv_layer)
        pool_layer_lst.append(maxpool_layer)
    #Dropout层
    pool_layer = tf.keras.layers.concatenate(pool_layer_lst, 3, name ="pool_layer")  
    max_num = len(window_sizes) * filter_num
    pool_layer_flat = tf.keras.layers.Reshape([1, max_num], name = "pool_layer_flat")(pool_layer)

    dropout_layer = tf.keras.layers.Dropout(dropout_keep, name = "dropout_layer")(pool_layer_flat)
    return pool_layer_flat, dropout_layer

def get_movie_feature_layer(movie_id_embed_layer, movie_categories_embed_layer, dropout_layer):
    #第一层全连接
    movie_id_fc_layer = tf.keras.layers.Dense(embed_dim, name="movie_id_fc_layer", activation='relu')(movie_id_embed_layer)
    movie_categories_fc_layer = tf.keras.layers.Dense(embed_dim, name="movie_categories_fc_layer", activation='relu')(movie_categories_embed_layer)

    #第二层全连接
    movie_combine_layer = tf.keras.layers.concatenate([movie_id_fc_layer, movie_categories_fc_layer, dropout_layer], 2)  
    movie_combine_layer = tf.keras.layers.Dense(200, activation='tanh')(movie_combine_layer)

    movie_combine_layer_flat = tf.keras.layers.Reshape([200], name="movie_combine_layer_flat")(movie_combine_layer)
    return movie_combine_layer, movie_combine_layer_flat

# 获取输入占位符
uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles = get_inputs()
# 获取User的4个嵌入向量
uid_embed_layer, gender_embed_layer, age_embed_layer, job_embed_layer = get_user_embedding(uid, user_gender, user_age, user_job)
# 得到用户特征
user_combine_layer, user_combine_layer_flat = get_user_feature_layer(uid_embed_layer, gender_embed_layer, age_embed_layer, job_embed_layer)


# 获取电影ID的嵌入向量
movie_id_embed_layer = get_movie_id_embed_layer(movie_id)
# 获取电影类型的嵌入向量
movie_categories_embed_layer = get_movie_categories_layers(movie_categories)
# 获取电影名的特征向量
pool_layer_flat, dropout_layer = get_movie_cnn_layer(movie_titles)
# 得到电影特征
movie_combine_layer, movie_combine_layer_flat = get_movie_feature_layer(movie_id_embed_layer, movie_categories_embed_layer, dropout_layer)

inference = tf.keras.layers.Lambda(lambda layer: tf.reduce_sum(layer[0] * layer[1], axis=1), name="inference")((user_combine_layer_flat, movie_combine_layer_flat))
inference = tf.keras.layers.Lambda(lambda layer: tf.expand_dims(layer, axis=1))(inference)
model = tf.keras.Model(
    inputs=[uid, user_gender, user_age, user_job, movie_id, movie_categories, movie_titles],
    outputs=[inference]
)
print(model.summary())
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# batch_size = batch_size
# best_loss = 9999
# losses = {'train': [], 'test': []}
# optimizer = tf.keras.optimizers.Adam(learning_rate)
# # MSE损失，将计算值回归到评分
# ComputeLoss = tf.keras.losses.MeanSquaredError()
# ComputeMetrics = tf.keras.metrics.MeanAbsoluteError()

Model: "model_22"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
movie_titles (InputLayer)       [(None, 15)]         0                                            
__________________________________________________________________________________________________
movie_title_embed_layer (Embedd (None, 15, 32)       166880      movie_titles[0][0]               
__________________________________________________________________________________________________
reshape (Reshape)               (None, 15, 32, 1)    0           movie_title_embed_layer[0][0]    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 14, 1, 8)     520         reshape[0][0]                    
___________________________________________________________________________________________

In [93]:
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(features, targets_values, test_size=0.2, random_state=0)
print(targets_values)
print(train_X.shape, test_X.shape, train_y.shape, test_y.shape)
print(test_X,'\n',test_y )
print(features_pd.columns)
print(targets_pd.columns)
print(len(features))
print([features.take(_,1) for _ in range(len(features))])
# history = model.fit([features.take(_,1) for _ in range(7)], targets_values, epochs=30, batch_size=batch_size, validation_split=0.2)

[[5]
 [5]
 [4]
 ...
 [1]
 [5]
 [4]]
(800167, 7) (200042, 7) (800167, 1) (200042, 1)
[[1996 2406 1 ... 0
  list([3283, 2924, 3742, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867])
  list([14, 15, 0, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])]
 [921 2071 1 ... 17
  list([2036, 2924, 1942, 2, 2647, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867])
  list([2, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])]
 [323 3809 1 ... 12
  list([4198, 3295, 2564, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867])
  list([0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])]
 ...
 [2886 2916 1 ... 4
  list([4821, 5005, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867])
  list([14, 15, 1, 10, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])]
 [1767 2985 1 ... 20
  list([4849, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867, 1867])
  list([14, 9, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 

IndexError: index 7 is out of bounds for axis 1 with size 7

In [ ]:
def compute_loss(labels, logits):
    return tf.reduce_mean(tf.keras.losses.mse(labels, logits))

def compute_metrics(self, labels, logits):
    return tf.keras.metrics.mae(labels, logits)  #

def get_batches(Xs, ys, batch_size):
    for start in range(0, len(Xs), batch_size):
        end = min(start + batch_size, len(Xs))
        yield Xs[start:end], ys[start:end]

@tf.function
def train_step(x, y):
    # Record the operations used to compute the loss, so that the gradient
    # of the loss with respect to the variables can be computed.
    #         metrics = 0
    with tf.GradientTape() as tape:
        logits = model([x[0],
                                x[1],
                                x[2],
                                x[3],
                                x[4],
                                x[5],
                                x[6]], training=True)
        loss = ComputeLoss(y, logits)
        # loss = compute_loss(labels, logits)
        ComputeMetrics(y, logits)
        # metrics = compute_metrics(labels, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss, logits
import time
def training(features, targets_values, epochs=5, log_freq=50):

    for epoch_i in range(epochs):
        # 将数据集分成训练集和测试集，随机种子不固定
        from sklearn.model_selection import train_test_split
        train_X, test_X, train_y, test_y = train_test_split(features,
                                                            targets_values,
                                                            test_size=0.2,
                                                            random_state=0)

        train_batches = get_batches(train_X, train_y, batch_size)
        batch_num = (len(train_X) // batch_size)

        train_start = time.time()
        #             with train_summary_writer.as_default():
        if True:
            start = time.time()
            # Metrics are stateful. They accumulate values and return a cumulative
            # result when you call .result(). Clear accumulated values with .reset_states()
            avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)
            #                 avg_mae = tf.keras.metrics.Mean('mae', dtype=tf.float32)

            # Datasets can be iterated over like any other Python iterable.
            for batch_i in range(batch_num):
                x, y = next(train_batches)
                categories = np.zeros([batch_size, 18])
                for i in range(batch_size):
                    categories[i] = x.take(6, 1)[i]

                titles = np.zeros([batch_size, sentences_size])
                for i in range(batch_size):
                    titles[i] = x.take(5, 1)[i]

                loss, logits = train_step([np.reshape(x.take(0, 1), [batch_size, 1]).astype(np.float32),
                                                np.reshape(x.take(2, 1), [batch_size, 1]).astype(np.float32),
                                                np.reshape(x.take(3, 1), [batch_size, 1]).astype(np.float32),
                                                np.reshape(x.take(4, 1), [batch_size, 1]).astype(np.float32),
                                                np.reshape(x.take(1, 1), [batch_size, 1]).astype(np.float32),
                                                categories.astype(np.float32),
                                                titles.astype(np.float32)],
                                                np.reshape(y, [batch_size, 1]).astype(np.float32))
                avg_loss(loss)
                #                     avg_mae(metrics)
                losses['train'].append(loss)

                if tf.equal(optimizer.iterations % log_freq, 0):
                    #                         summary_ops_v2.scalar('loss', avg_loss.result(), step=optimizer.iterations)
                    #                         summary_ops_v2.scalar('mae', ComputeMetrics.result(), step=optimizer.iterations)
                    # summary_ops_v2.scalar('mae', avg_mae.result(), step=optimizer.iterations)

                    rate = log_freq / (time.time() - start)
                    print('Step #{}\tEpoch {:>3} Batch {:>4}/{}   Loss: {:0.6f} mae: {:0.6f} ({} steps/sec)'.format(
                        optimizer.iterations.numpy(),
                        epoch_i,
                        batch_i,
                        batch_num,
                        loss, (ComputeMetrics.result()), rate))
                    # print('Step #{}\tLoss: {:0.6f} mae: {:0.6f} ({} steps/sec)'.format(
                    #     optimizer.iterations.numpy(), loss, (avg_mae.result()), rate))
                    avg_loss.reset_states()
                    ComputeMetrics.reset_states()
                    # avg_mae.reset_states()
                    start = time.time()

        train_end = time.time()
        print(
            '\nTrain time for epoch #{} ({} total steps): {}'.format(epoch_i + 1, optimizer.iterations.numpy(),
                                                                        train_end - train_start))
        #             with test_summary_writer.as_default():
        testing((test_X, test_y), optimizer.iterations)
        # checkpoint.save(checkpoint_prefix)
    export_path = os.path.join(MODEL_DIR, 'export')
    tf.saved_model.save(model, export_path)

def testing(test_dataset, step_num):
    test_X, test_y = test_dataset
    test_batches = get_batches(test_X, test_y, batch_size)

    """Perform an evaluation of `model` on the examples from `dataset`."""
    avg_loss = tf.keras.metrics.Mean('loss', dtype=tf.float32)
    #         avg_mae = tf.keras.metrics.Mean('mae', dtype=tf.float32)

    batch_num = (len(test_X) // batch_size)
    for batch_i in range(batch_num):
        x, y = next(test_batches)
        categories = np.zeros([batch_size, 18])
        for i in range(batch_size):
            categories[i] = x.take(6, 1)[i]

        titles = np.zeros([batch_size, sentences_size])
        for i in range(batch_size):
            titles[i] = x.take(5, 1)[i]

        logits = model([np.reshape(x.take(0, 1), [batch_size, 1]).astype(np.float32),
                                np.reshape(x.take(2, 1), [batch_size, 1]).astype(np.float32),
                                np.reshape(x.take(3, 1), [batch_size, 1]).astype(np.float32),
                                np.reshape(x.take(4, 1), [batch_size, 1]).astype(np.float32),
                                np.reshape(x.take(1, 1), [batch_size, 1]).astype(np.float32),
                                categories.astype(np.float32),
                                titles.astype(np.float32)], training=False)
        test_loss = ComputeLoss(np.reshape(y, [batch_size, 1]).astype(np.float32), logits)
        avg_loss(test_loss)
        # 保存测试损失
        losses['test'].append(test_loss)
        ComputeMetrics(np.reshape(y, [batch_size, 1]).astype(np.float32), logits)
        # avg_loss(compute_loss(labels, logits))
        # avg_mae(compute_metrics(labels, logits))

    print('Model test set loss: {:0.6f} mae: {:0.6f}'.format(avg_loss.result(), ComputeMetrics.result()))
    # print('Model test set loss: {:0.6f} mae: {:0.6f}'.format(avg_loss.result(), avg_mae.result()))
    #         summary_ops_v2.scalar('loss', avg_loss.result(), step=step_num)
    #         summary_ops_v2.scalar('mae', ComputeMetrics.result(), step=step_num)
    # summary_ops_v2.scalar('mae', avg_mae.result(), step=step_num)

    if avg_loss.result() < best_loss:
        best_loss = avg_loss.result()
        print("best loss = {}".format(best_loss))
        checkpoint.save(checkpoint_prefix)

def forward(xs):
    predictions = model(xs)
    # logits = tf.nn.softmax(predictions)

In [ ]:
training(features, targets_values, epochs=5)